In [1]:
import logging
import os
from fetch_data import *
from forward_integration import *
from initial_states import *
import pandas as pd
import numpy as np

curr_dir = os.getcwd()
stats_dir = os.path.join(curr_dir, 'stats')
logger = logging.getLogger(__name__)
erva_pop_file = os.path.join(stats_dir, 'erva_population_age_2020.csv')
print(curr_dir)

/Users/aponcedeleonch/Documents/Aalto/Courses/MasterThesis/Code/jeta


In [2]:
infectios_age_finland, infectios_age_finland_prop = fetch_finland_cases_age_weekly(logger, 9)
infectios_age_finland.loc[infectios_age_finland['Time'] == 'Year 2021 Week 15']

,Time,age_group,val
670,Year 2021 Week 15,0-9,240.0
671,Year 2021 Week 15,10-19,310.0
672,Year 2021 Week 15,20-29,354.0
673,Year 2021 Week 15,30-39,355.0
674,Year 2021 Week 15,40-49,294.0
675,Year 2021 Week 15,50-59,200.0
676,Year 2021 Week 15,60-69,101.0
677,Year 2021 Week 15,70-79,44.0
678,Year 2021 Week 15,80+,31.0


In [3]:
infectios_age_finland_prop.loc[infectios_age_finland_prop['Time'] == 'Year 2021 Week 15']

,Time,age_group,val
670,Year 2021 Week 15,0-9,0.124417
671,Year 2021 Week 15,10-19,0.160705
672,Year 2021 Week 15,20-29,0.183515
673,Year 2021 Week 15,30-39,0.184033
674,Year 2021 Week 15,40-49,0.152411
675,Year 2021 Week 15,50-59,0.103681
676,Year 2021 Week 15,60-69,0.052359
677,Year 2021 Week 15,70-79,0.022810
678,Year 2021 Week 15,80+,0.016071


In [4]:
rounded_vals = infectios_age_finland_prop.loc[infectios_age_finland_prop['Time'] == 'Year 2021 Week 15'].val
np.round(rounded_vals, 4)

670    0.1244
671    0.1607
672    0.1835
673    0.1840
674    0.1524
675    0.1037
676    0.0524
677    0.0228
678    0.0161
Name: val, dtype: float64

In [5]:
hosp_by_erva = fetch_hs_hospitalizations(logger)
hosp_by_erva.loc[hosp_by_erva['date'] == '2021-04-19', ]

,date,erva,hospitalized,ward,icu,dead
1481,2021-04-19,HYKS,109,88,21,578
1482,2021-04-19,KYS,10,5,5,81
1483,2021-04-19,OYS,11,11,0,55
1484,2021-04-19,TAYS,19,17,2,79
1485,2021-04-19,TYKS,17,11,6,98


In [6]:
cases_erva = fetch_thl_cases_erva_daily(logger)

In [7]:
cases_erva.loc[(cases_erva['Time'] >= '2021-04-12') & (cases_erva['Time'] <= '2021-04-18')].groupby('erva').sum()

,val
erva,
HYKS,1179.0
KYS,81.0
OYS,75.0
TAYS,225.0
TYKS,346.0
Åland,23.0


In [8]:
t0 = '2021-04-18'
number_age_groups = 9
num_ervas = 5
erva_pop_file = 'stats/erva_population_age_2020.csv'

pop_ervas_age, _ = static_population_erva_age(logger, erva_pop_file,
                                              number_age_groups=number_age_groups)
pop_ervas_age = pop_ervas_age[~pop_ervas_age['erva'].str.contains('All')]
pop_ervas_age = pop_ervas_age[~pop_ervas_age['erva'].str.contains('land')]
pop_ervas_age = pop_ervas_age.sort_values(['erva', 'age_group'])
pop_ervas_npy = pop_ervas_age['Total'].values
pop_ervas_npy = pop_ervas_npy.reshape(num_ervas, number_age_groups)

ervas_order = EPIDEMIC['ervas_order']

ervas_df = list(pd.unique(pop_ervas_age['erva']))
ervas_pd_order = [ervas_df.index(erva) for erva in ervas_order]
# Rearrange rows in the correct order
age_er = pop_ervas_npy[ervas_pd_order, :]

N_p = num_ervas
N_g = number_age_groups
csv_name = 'out/epidemic_finland_%d.csv' % (number_age_groups, )

# Reading CSV
epidemic_csv = pd.read_csv(csv_name)
# Getting only date t0
epidemic_zero = epidemic_csv.loc[epidemic_csv['date'] == t0, :]
# Removing Ahvenanmaa or Aland
epidemic_zero = epidemic_zero[~epidemic_zero['erva'].str.contains('land')]

# Getting the order the ervas have inside the dataframe
ervas_order = EPIDEMIC['ervas_order']
ervas_df = list(pd.unique(epidemic_zero['erva']))
ervas_pd_order = [ervas_df.index(erva) for erva in ervas_order]

select_columns = ['susceptible',
                  'infected',
                  'exposed',
                  'recovered',
                  'vaccinated',
                  'ward',
                  'icu',
                  'infected detected',
                  'infected undetected',
                  'First dose cumulative',]
# Selecting the columns to use
epidemic_zero = epidemic_zero[select_columns]
# Converting to numpy
epidemic_npy = epidemic_zero.values
# Reshaping to 3d array
epidemic_npy = epidemic_npy.reshape(N_p, N_g, len(select_columns))

# Rearranging the order of the matrix with correct order
epidemic_npy = epidemic_npy[ervas_pd_order, :]

In [9]:
def print_latex(compartment):
    assert compartment.shape == (N_p, N_g)
    
    for age_i in range(N_g):
        for erva_i in range(N_p):
            if erva_i + 1 == N_p:
                total = np.sum(compartment[:, age_i])
                total_age = (total/np.sum(age_er[:, age_i]))*100
                print(' %.2f & %.2f & %.2f \\\\' % (round(compartment[erva_i, age_i], 2),
                                                   round(total, 2),
                                                   round(total_age, 2)), end='')
            else:
                print(' %.2f &' % (round(compartment[erva_i, age_i], 2), ), end='')
        print('')
    
    print('')
    sum_ervas = compartment.sum(axis=1)
    for erva_i in range(N_p):
        print(' %.2f &' % (round(sum_ervas[erva_i], 2), ), end='')
        if erva_i + 1 == N_p:
            print(' foo & foo \\\\')
    
    print('')
    prop_ervas = (sum_ervas/age_er.sum(axis=1))*100
    for erva_i in range(N_p):
        print(' %.2f &' % (round(prop_ervas[erva_i], 2), ), end='')
        if erva_i + 1 == N_p:
            print(' foo & \\\\')
    
    total = np.sum(compartment)
    total_prop = (total/np.sum(age_er))*100
    print("%.2f & %.2f" % (total, total_prop))

In [10]:
# v_kg
print_latex(epidemic_npy[:, :, 9])

 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 \\
 1802.00 & 895.00 & 647.00 & 467.00 & 397.00 & 4208.00 & 0.69 \\
 14326.00 & 6391.00 & 4806.00 & 4111.00 & 3570.00 & 33204.00 & 5.04 \\
 22284.00 & 8958.00 & 7639.00 & 6314.00 & 5640.00 & 50835.00 & 7.11 \\
 32713.00 & 12418.00 & 11718.00 & 8261.00 & 7842.00 & 72952.00 & 11.01 \\
 53123.00 & 20671.00 & 20143.00 & 15545.00 & 14676.00 & 124158.00 & 17.47 \\
 111319.00 & 46461.00 & 47329.00 & 40640.00 & 33953.00 & 279702.00 & 39.54 \\
 184419.00 & 87350.00 & 85498.00 & 79872.00 & 63631.00 & 500770.00 & 85.85 \\
 94809.00 & 50321.00 & 49239.00 & 47561.00 & 37125.00 & 279055.00 & 88.96 \\

 514795.00 & 233465.00 & 227019.00 & 202771.00 & 166834.00 & foo & foo \\

 23.42 & 26.87 & 25.15 & 25.43 & 22.65 & foo & \\
1344884.00 & 24.44


In [11]:
# S^u_kg
print_latex(epidemic_npy[:, :, 0])

 171581.02 & 70816.88 & 81903.12 & 67257.71 & 76718.73 & 468277.46 & 85.97 \\
 209955.74 & 85708.42 & 96837.14 & 80990.48 & 88887.55 & 562379.33 & 92.52 \\
 228534.80 & 89876.46 & 96665.64 & 85189.05 & 78392.50 & 578658.45 & 87.88 \\
 270855.73 & 91783.82 & 102310.14 & 82761.97 & 80860.11 & 628571.77 & 87.92 \\
 235429.06 & 85427.65 & 93018.80 & 75258.19 & 73005.88 & 562139.57 & 84.84 \\
 221934.99 & 88013.76 & 93279.21 & 86411.78 & 75323.60 & 564963.33 & 79.49 \\
 137785.90 & 65351.37 & 69714.24 & 78372.62 & 65760.56 & 416984.69 & 58.95 \\
 24360.73 & 11823.42 & 14133.96 & 15379.26 & 11743.93 & 77441.30 & 13.28 \\
 8689.93 & 5505.76 & 6064.27 & 5424.06 & 4885.78 & 30569.80 & 9.75 \\

 1509127.92 & 594307.53 & 653926.52 & 577045.10 & 555578.63 & foo & foo \\

 68.65 & 68.39 & 72.44 & 72.38 & 75.43 & foo & \\
3889985.71 & 70.68


In [12]:
# i^r_kg
print_latex(epidemic_npy[:, :, 7] + epidemic_npy[:, :, 8])

 1613.56 & 473.53 & 307.93 & 110.86 & 102.64 & 2608.52 & 0.48 \\
 688.86 & 202.16 & 131.46 & 47.33 & 43.82 & 1113.63 & 0.18 \\
 563.26 & 165.30 & 107.49 & 38.70 & 35.83 & 910.58 & 0.14 \\
 498.45 & 146.28 & 95.12 & 34.24 & 31.71 & 805.81 & 0.11 \\
 390.24 & 114.52 & 74.47 & 26.81 & 24.82 & 630.87 & 0.10 \\
 257.88 & 75.68 & 49.21 & 17.72 & 16.40 & 416.90 & 0.06 \\
 128.09 & 37.59 & 24.45 & 8.80 & 8.15 & 207.08 & 0.03 \\
 55.24 & 16.21 & 10.54 & 3.80 & 3.51 & 89.30 & 0.02 \\
 38.66 & 11.35 & 7.38 & 2.66 & 2.46 & 62.50 & 0.02 \\

 4234.25 & 1242.62 & 808.06 & 290.90 & 269.35 & foo & foo \\

 0.19 & 0.14 & 0.09 & 0.04 & 0.04 & foo & \\
6845.19 & 0.12


In [13]:
# r^r_kg
print_latex(epidemic_npy[:, :, 3])

 48417.83 & 11521.51 & 5859.84 & 4541.39 & 3486.56 & 73827.12 & 13.55 \\
 25865.31 & 6195.26 & 3248.20 & 2708.11 & 2142.51 & 40159.39 & 6.61 \\
 29245.20 & 7138.55 & 3695.02 & 3126.87 & 2512.15 & 45717.79 & 6.94 \\
 22528.67 & 5204.00 & 2763.63 & 2279.29 & 1915.51 & 34691.10 & 4.85 \\
 17445.63 & 4017.24 & 2137.99 & 1755.06 & 1474.30 & 26830.21 & 4.05 \\
 13794.68 & 3110.63 & 1682.66 & 1410.67 & 1207.40 & 21206.04 & 2.98 \\
 6753.32 & 1529.61 & 825.10 & 699.36 & 598.59 & 10405.98 & 1.47 \\
 3231.22 & 723.69 & 398.59 & 333.72 & 288.35 & 4975.57 & 0.85 \\
 2633.98 & 531.37 & 297.38 & 262.53 & 244.63 & 3969.89 & 1.27 \\

 169915.83 & 39971.85 & 20908.42 & 17117.00 & 13870.01 & foo & foo \\

 7.73 & 4.60 & 2.32 & 2.15 & 1.88 & foo & \\
261783.10 & 4.76
